# ASSIGNMENT 2
## PART 1: MOVIE REVIEW DATASET V1.0

### Overview
Dữ liệu được cung cấp bao gồm 2 thư mục trong đó là các review bằng file txt. Dữ liệu được phân vùng sẵn trong các thư mục pos (review tích cực) và neg (review tiêu cực hay review xấu)
Ngoài 2 kiểu review này sẽ có 1 thư mục dạng `unsup` là loại review chưa được xử lý và cần định dạng theo kiểu tích cực hay tiêu cực.
Các file txt này được định dạng tên như sau:
thứ tự_điểm số.
Để chi tiết hơn tham khảo [README](aclImdb/README)
For more detail refer to [README](aclImdb/README)

### 1. Mô tả về Assigment
Dữ liệu sẽ được load vào 4 datasets:
   * negative_test_data: Test Data of Negative Reviews,
   * positive_test_data: Test Data of Positive Reviews,
   * negative_train_data: Train Data of Nagative Reviews,
   * positive_train_data: Train Data of Positive Reviews,
   * The unsup data is unsupervised data for apply the appropriate model,
 

### 2. Nhiệm vụ (Task List)
   - [1 ] IMPORT DATA
   - [2] FEATURING DATA
   - [3] MODELING THE DATA BY TRAIN & TEST DATA
   - [4] USE MODEL TO ANLYZE THE UNSUP DATA
   - [5] CHECK THE CORRECTION OF PREDICTION MODEL

In [90]:
# Load the library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#### 2.1. Tải dữ liệu vào DataFrame
Bên trong thư mục, các file được định dạng `[[id]_[rating].txt]` tại đó `[id]` là `unique id` và `[rating]` là `star rating` với dải điểm số từ 1 - 10,
Bốn (4) dataset sẽ được xử lý như sau:
* negative_test_data: Test Data of Negative Reviews,
* positive_test_data: Test Data of Positive Reviews,
* negative_train_data: Train Data of Nagative Reviews,
* positive_train_data: Train Data of Positive Reviews,
* The unsup data is unsupervised data for apply the appropriate model

Do việc load dataset quá chậm, module ***data_collector*** được tạo ra mục đích để thu thập hết số liệu trước vào các file csv
*Note: The load() function of sklearn is failed to collect the data*

In [91]:
negative_train_data = pd.read_csv('neg_train_data.csv', header=None, index_col=0)
negative_train_data.columns = ['ID', 'Rating', 'Content of Review']

positive_train_data = pd.read_csv('pos_train_data.csv', header=None, index_col=0)
positive_train_data.columns = ['ID', 'Rating', 'Content of Review']

negative_test_data = pd.read_csv('neg_test_data.csv', header=None, index_col=0)
negative_test_data.columns = ['ID', 'Rating', 'Content of Review']

positive_test_data = pd.read_csv('pos_test_data.csv', header=None, index_col=0)
positive_test_data.columns = ['ID', 'Rating', 'Content of Review']

unsupervised_data = pd.read_csv('unsup_data.csv', header=None, index_col=0)
unsupervised_data.columns = ['ID', 'Rating', 'Content of Review']

#### 2.2. Tập hợp dữ liệu và nhãn hóa (Labeled)

In [92]:
sentiment_train_data = pd.concat([negative_train_data, positive_train_data])
sentiment_test_data = pd.concat([negative_test_data, positive_test_data])

In [93]:
print(sentiment_train_data.shape)
sentiment_train_data.head()

(25000, 3)


,ID,Rating,Content of Review
0,,,
0,0,3,Story of a man who has unnatural feelings for ...
1,10000,4,Airport '77 starts as a brand new luxury 747 p...
2,10001,4,This film lacked something I couldn't put my f...
3,10002,1,"Sorry everyone,,, I know this is supposed to b..."
4,10003,1,When I was little my parents took me along to ...


In [94]:
print(sentiment_test_data.shape)
sentiment_test_data.head(10)

(25000, 3)


,ID,Rating,Content of Review
0,,,
0,0,2,Once again Mr. Costner has dragged out a movie...
1,10000,4,This is an example of why the majority of acti...
2,10001,1,"First of all I hate those moronic rappers, who..."
3,10002,3,Not even the Beatles could write songs everyon...
4,10003,3,Brass pictures (movies is not a fitting word f...
5,10004,2,"A funny thing happened to me while watching ""M..."
6,10005,2,This German horror film has to be one of the w...
7,10006,2,"Being a long-time fan of Japanese film, I expe..."
8,10007,4,"""Tokyo Eyes"" tells of a 17 year old Japanese g..."


##### Check info của `upsup` data. Những dữ liệu này đã được labeled

In [95]:
print(unsupervised_data.info())
unsupervised_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 50000 non-null  int64 
 1   Rating             50000 non-null  int64 
 2   Content of Review  50000 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
None


,ID,Rating,Content of Review
0,,,
0,0,0,"I admit, the great majority of films released ..."
1,10000,0,"Take a low budget, inexperienced actors doubli..."
2,10001,0,"Everybody has seen 'Back To The Future,' right..."
3,10002,0,Doris Day was an icon of beauty in singing and...
4,10003,0,"After a series of silly, fun-loving movies, 19..."


#### 2.3. Tiền xử lý dữ liệu (Pre-progresssing Data)
Sau khi chạy thử các mô hình CountVectorizer và TfIdf cho dữ liệu thô kết quả cho độ chính xác như sau:
CountVectorizer: 68%
Tfiđf: 59%
Cả hai độ chính xác này đều không đạt tiêu chuẩn cần thiết theo yêu cầu đề bài, do đó việc chuẩn hóa lại dữ liệu là cần thiết (tiền xử lý dữ liệu lại)

In [96]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vutua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vutua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Tiền xử lý văn bản gồm các thao tác:
* Chuyển hết về ký tự dạng thấp
* Gỡ bỏ các stop word
* Xử lý một số dấu ký tự đặc biệt

In [116]:
import re
from nltk.tokenize import word_tokenize

def preprocessed_text(text):
    """
    Preprocess the input as text, clean all Html tags and others, lower text and remove punctuation
    :param text: 
    :return: the clean text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r"'", '', text)
    text = re.sub(r'\\', '', text)
    text = re.sub(r'"', '', text)
    
    pattern = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n~``'
    # text = re.sub(pattern, '', text)
    
    text = text.strip().lower()
    
    # Tách từ
    # text_tokenized = word_tokenize(text)
    
    # filter_text = [word for word in text_tokenized if word not in pattern]
    # return ' '.join(filter_text)
    translate_dict = dict((c, " ") for c in pattern)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    return text
    
# Example
txt1 = sentiment_train_data['Content of Review'].values[4]
print(txt1)
print(preprocessed_text(txt1))

When I was little my parents took me along to the theater to see Interiors. It was one of many movies I watched with my parents, but this was the only one we walked out of. Since then I had never seen Interiors until just recently, and I could have lived out the rest of my life without it. What a pretentious, ponderous, and painfully boring piece of 70's wine and cheese tripe. Woody Allen is one of my favorite directors but Interiors is by far the worst piece of crap of his career. In the unmistakable style of Ingmar Berman, Allen gives us a dark, angular, muted, insight in to the lives of a family wrought by the psychological damage caused by divorce, estrangement, career, love, non-love, halitosis, whatever. The film, intentionally, has no comic relief, no music, and is drenched in shadowy pathos. This film style can be best defined as expressionist in nature, using an improvisational method of dialogue to illicit a "more pronounced depth of meaning and truth". But Woody Allen is no 

##### Chuẩn hóa lại dữ liệu

In [117]:
sentiment_train_data['Preprocessed Data'] = sentiment_train_data['Content of Review'].map(preprocessed_text)
sentiment_test_data['Preprocessed Data'] = sentiment_test_data['Content of Review'].map(preprocessed_text)

In [118]:
print(sentiment_test_data.shape)
sentiment_train_data['Preprocessed Data'][:5]

(25000, 4)


0
0    story of a man who has unnatural feelings for ...
1    airport 77 starts as a brand new luxury 747 pl...
2    this film lacked something i couldnt put my fi...
3    sorry everyone    i know this is supposed to b...
4    when i was little my parents took me along to ...
Name: Preprocessed Data, dtype: object

### 3. Đặc trưng hóa dữ liệu (Featuring the data)
Nội dung Content of Review sẽ được vector hóa phục vụ cho việc xây dựng model
Để huấn luyện mô hình, áp dụng thử hai loại chuẩn hóa là CountVectorizer và Tf-idf để đánh giá sai số   

#### 3.1 Chuyển đổi Vector sử dụng CountVectorizer

In [150]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english', preprocessor=preprocessed_text)

# Tập huấn luyện - chú ý dữ liệu đầu vào cần để 1 mảng hoặc tách bằng tham số word_tokenize
Xtrain = vectorizer.fit_transform(sentiment_train_data['Content of Review'])
ytrain = sentiment_train_data['Rating'].astype(int).values

Xtest = vectorizer.transform(sentiment_test_data['Content of Review'])
ytest = sentiment_test_data['Rating'].astype(int).values

#### 3.2 Chuyển đổi Vector sử dụng Tf-idf

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', preprocessor=preprocessed_text)

Xtrain_tfidf = tfidf_vectorizer.fit_transform(sentiment_train_data['Preprocessed Data'])
Xtest_tfidf = tfidf_vectorizer.transform(sentiment_test_data['Preprocessed Data'])

### 4. Áp dụng Logistics Regression cho việc dự đoán và tìm sai số

##### Sử dụng mô hình SVC với bước 1 cho dữ liệu của CountVectorizer

In [151]:
from sklearn.svm import LinearSVC

lrSVC_model = LinearSVC(max_iter=5000).fit(Xtrain, ytrain)

##### Thử nghiệm dự đoán và sai số với CountVectorizer

In [152]:
ypred_cr = lrSVC_model.predict(Xtest)

from sklearn.metrics import accuracy_score
accuracy_score(ytest, ypred_cr)

0.3204

##### Thử nghiệm dự đoán và sai số với bước 2 dùng tf-idf

In [153]:
lrSVC_tfidf = LinearSVC(max_iter=5000).fit(Xtrain_tfidf, ytrain)
ypred_tfidf = lrSVC_tfidf.predict(Xtest_tfidf)
accuracy_score(ytest, ypred_tfidf)

0.37196

#### *KẾT LUẬN*
Đối với dự đoán kết quả rating sử dụng phương án hồi quy cho kết quả độ chính xác rất thấp với 
**CountVectorizer**: 32% và **Tf-Idf**: 37%

### 5. Trường hợp chỉ chuẩn đoán tích cực hay tiêu cực
Nếu coi pos là 1 và neg là 0. Tạo một bảng dữ liệu khác với 2 cột là Content of Review như features và label sẽ là 0 hoặc 1.
Với phần này sử dụng tên với tag *`r_`* để thực hiện: `rt - train data, rs - test data` nhằm không bị lẫn với phần 4

#### 5.1 Tạo các database của train và test

In [144]:
# Create dataframe of train
rt_positive_frame = pd.DataFrame({'Content of Review': positive_train_data['Content of Review'], 'rating': [1 for _ in range(len(positive_train_data['Content of Review']))]})
rt_negative_frame = pd.DataFrame({'Content of Review': negative_train_data['Content of Review'], 'rating': [0 for _ in range(len(negative_train_data['Content of Review']))]})

r_sensitive_train_df = pd.concat([rt_positive_frame, rt_negative_frame]).reset_index(drop=True)
r_sensitive_train_df[12490:12600]

,Content of Review,rating
12490,The most hillarious and funny Brooks movie I e...,1
12491,"""Life stinks"" is a parody of life and death, h...",1
12492,This is the kind of film you want to see with ...,1
12493,I have not read the other comments on the film...,1
12494,Life Stinks (1991) was a step below Mel Brooks...,1
...,...,...
12595,I remember watching this film in the eighties ...,0
12596,"This is probably the first entry in the ""Lance...",0
12597,"I am an actor,producer, director and what i am...",0
12598,German filmmaker Ulli Lommel has managed a tas...,0


In [145]:
# Create dataframe of test
rs_positive_frame = pd.DataFrame({'Content of Review': positive_test_data['Content of Review'], 'rating': [1 for _ in range(len(positive_test_data['Content of Review']))]})
rs_negative_frame = pd.DataFrame({'Content of Review': negative_test_data['Content of Review'], 'rating': [0 for _ in range(len(negative_test_data['Content of Review']))]})

r_sensitive_test_df = pd.concat([rs_positive_frame, rs_negative_frame]).reset_index(drop=True)
r_sensitive_test_df[12490:12600]

,Content of Review,rating
12490,if you watch this at home on DVD or Bluray! be...,1
12491,I'm not a sports fan - but I love sports flics...,1
12492,"As you know ""The Greatest Game Ever Played"" is...",1
12493,"This movie gives golf a high mark, it was well...",1
12494,"Wow, here is another great golf movie. That's ...",1
...,...,...
12595,Tyra Banks needs to teach these girls that it'...,0
12596,"This is by far the most vapid, idiotic, insane...",0
12597,It was awful plain and simple. What was their ...,0
12598,Wow! i think they made this movie to torture p...,0


In [157]:
print(r_sensitive_train_df['Content of Review'].shape, r_sensitive_train_df['rating'].shape)

(25000,) (25000,)


#### 5.2 Vectorize bằng CountVectorizer và tính sai số dựa trên LinerSVC

In [155]:
r_vectorizer = CountVectorizer(stop_words='english', preprocessor=preprocessed_text)

r_Xtrain = r_vectorizer.fit_transform(r_sensitive_train_df['Content of Review'])
r_ytrain = r_sensitive_train_df['rating'].astype(int).values

r_Xtest = r_vectorizer.transform(r_sensitive_test_df['Content of Review'])
r_ytest = r_sensitive_test_df['rating'].astype(int).values

##### Thực hiện bước 1 với CountVectorizer

In [159]:
r_lrSVC_model = LinearSVC(max_iter=5000).fit(r_Xtrain, r_ytrain)
r_ypred_cr = r_lrSVC_model.predict(r_Xtest)

accuracy = accuracy_score(r_ytest, r_ypred_cr)
accuracy

0.83676

##### Thực hiện bước 2 với tf-idf

In [160]:
r_tfidf_vectorizer = TfidfVectorizer(stop_words='english', preprocessor=preprocessed_text)

r_Xtrain_tfidf = tfidf_vectorizer.fit_transform(sentiment_train_data['Preprocessed Data'])
r_Xtest_tfidf = tfidf_vectorizer.transform(sentiment_test_data['Preprocessed Data'])

In [162]:
r_lrSVC_model = LinearSVC(max_iter=5000).fit(r_Xtrain_tfidf, r_ytrain)
r_ypred_tfidf = r_lrSVC_model.predict(r_Xtest_tfidf)

accuracy_tfidf = accuracy_score(r_ytest, r_ypred_tfidf)
accuracy_tfidf

0.86636

#### *KẾT LUẬN TRƯỜNG HỢP SAU*
Đối với dự đoán kết quả chỉ là negative hay positive sử dụng phương án hồi quy cho kết quả độ chính xác rất khá cao với 
**CountVectorizer**: ~84% và **Tf-Idf**: ~86%

#### 6. Tính sai số sử dụng F1 Score

In [168]:
from sklearn.metrics import f1_score

print('CountVectorizer:{f}'.format(f=f1_score(r_ytest, r_ypred_cr)), '\nTfIdf:{f}'.format(f=f1_score(r_ytest, r_ypred_tfidf)))

CountVectorizer:0.8349577385044687 
TfIdf:0.867929003439143


#### Kết luận:
Kết quả với F1 Score không có gì khác biệt với **83.4%** cho **CountVectorizer** và **87%** cho **TfIdf**